In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from blooms_ml.utils import (
    get_stats,
    labeling,
)

In [ ]:
sns.set_style("whitegrid")

Open the prepared from ROHO800 model output data and label it

In [ ]:
datadir = f"{Path.home()}/data_ROHO"
df = pd.read_parquet(f"{datadir}/roho800_weekly_average.parquet")
(p1_c_mean, n1_p_mean, n3_n_mean, n5_s_mean,
 p1_c_std, n1_p_std, n3_n_std, n5_s_std) = get_stats(f"{datadir}/cnps_mean_std.csv")

In [ ]:
df = df.groupby(['station', 's_rho']).apply(labeling, include_groups=False)
df = df.reset_index().drop(columns='level_2')

In [ ]:
df.rename(columns={'label': 'y'}, inplace=True)
df['label'] = np.where(df['y'] > 3, 1, 0)

In [ ]:
df = df[df['y'].notna()]

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df['y'].clip(lower=-30, upper=30), kde=True, color='g', bins=30)
plt.title('Histogram and KDE')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.show()

In [ ]:
df

"Normalization"

In [ ]:
df_norm = df  # .copy()

In [ ]:
df_norm['P1_c'] = ((df_norm['P1_c'] - float(p1_c_mean)) / float(p1_c_std)).round(2).astype('float32')
df_norm['N1_p'] = ((df_norm['N1_p'] - float(n1_p_mean)) / float(n1_p_std)).round(2).astype('float32')
df_norm['N3_n'] = ((df_norm['N3_n'] - float(n3_n_mean)) / float(n3_n_std)).round(2).astype('float32')
df_norm['N5_s'] = ((df_norm['N5_s'] - float(n5_s_mean)) / float(n5_s_std)).round(2).astype('float32')

In [ ]:
df_norm

Plot a station

In [ ]:
df_station = df[df['station'] == 0][df['s_rho'] == -0.02]
df_station.set_index('ocean_time', inplace=True)
df_station

In [ ]:
plot_year = '2008'
df_year = df_station.loc[plot_year]
df_label_year = df_year[df_year['label'] == 1]

In [ ]:
def plot_year(par):
    _, ax = plt.subplots(figsize=(20, 3))
    ax.plot(df_year[par].index, df_year[par])
    ax.plot(df_label_year.index, df_label_year[par], 'r.')

In [ ]:
plot_year('P1_c')

In [ ]:
plot_year('N5_s')